# **Homework 6 - Generative Adversarial Network**

This is the example code of homework 6 of the machine learning course by Prof. Hung-yi Lee.


In this homework, you are required to build a generative adversarial  network for anime face generation.


## Reference

For this homework I refered to lucidrains' GitHub directory for the simple StyleGAN2.


I followed the instructions written in the README file and performed minor tweaks to satisfy the requirements of this homework.


Repository Link:https://github.com/lucidrains/stylegan2-pytorch

## Set up the environment


### Checking for Tesla V100 GPU

In [ ]:
!nvidia-smi

Fri May  7 08:24:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Mount google drive to save checkpoints

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Packages Installation

In [ ]:
# You may replace the workspace directory if you want.
workspace_dir = '.'

# Training progress bar
!pip install -q qqdm

# Install stylegan2 & pytorch-fid
!pip install stylegan2_pytorch pytorch
!pip install pytorch-fid

     |████████████████████████████████| 276kB 16.0MB/s 
     |████████████████████████████████| 163kB 28.5MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
     |████████████████████████████████| 153kB 29.3MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 296kB 31.7MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 163kB 31.1MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
  Created wheel for vector-quantize-pytorch: filename=vector_quantize_pytorch-0.1.0-cp37-none-any.whl size=2576 sha256=b9fc12c5b5e5341a180781be66583cde2b5bc3bfb4c97993db0e9a6845920d57
  Stored in directory: /root/.cache/pip/wheels/10/68/a9/2eb0d1f485a2babf71ac5e471c9d45703cf2a2cc8686f780d2
  Created wheel for fire:

### Download the dataset
**Please use the link according to the last digit of your student ID first!**

If all download links fail, please follow [here](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e).

* To open the file using your browser, use the link below (replace the id first!):
https://drive.google.com/file/d/REPLACE_WITH_ID
* e.g. https://drive.google.com/file/d/1IGrTr308mGAaCKotpkkm8wTKlWs9Jq-p

In [ ]:
# Download dataset
!gdown --id 1GDT5b7b_6ur5JPFqljiZ_OyKEgrsZgrN --output "{workspace_dir}/crypko_data.zip"

Downloading...
From: https://drive.google.com/uc?id=1GDT5b7b_6ur5JPFqljiZ_OyKEgrsZgrN
To: /content/crypko_data.zip
452MB [00:09, 46.6MB/s]


###Unzip the downloaded file.
The unzipped tree structure is like 
```
faces/
├── 1.jpg
├── 2.jpg
├── 3.jpg
...
```

In [ ]:
!unzip -q "{workspace_dir}/crypko_data.zip" -d "{workspace_dir}/"

## Random seed
Set the random seed to a certain value for reproducibility.

In [ ]:
import random
import torch
import numpy as np

def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2021)

## Import Packages
First, we need to import packages that will be used later.

Like hw3, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
import os
import glob

import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from qqdm.notebook import qqdm
from stylegan2_pytorch import ModelLoader

## Training

Execute the training command and set hyperparameters.

In [ ]:
!stylegan2_pytorch \
--data /content/faces \
--name HW6_model \
--results_dir /content/drive/MyDrive/ML_HW6/results \
--models_dir /content/drive/MyDrive/ML_HW6/models \
--attn-layers 1 \
--num-train-steps 40000 \
--image-size 64

HW6_model</content/faces>:   0% 0/40000 [00:00<?, ?it/s]G: 754.70 | D: 36.54 | GP: 35.80
HW6_model</content/faces>:   0% 40/40000 [00:30<8:43:41,  1.27it/s]G: 78.89 | D: 0.00 | GP: 12.20
HW6_model</content/faces>:   0% 99/40000 [01:12<8:06:21,  1.37it/s]G: 21.64 | D: 1.87 | GP: 0.29
HW6_model</content/faces>:   0% 143/40000 [01:44<8:01:08,  1.38it/s]G: 3.87 | D: 1.29 | GP: 3.25
HW6_model</content/faces>:   0% 187/40000 [02:15<7:53:32,  1.40it/s]G: -0.48 | D: 2.35 | GP: 0.08
HW6_model</content/faces>:   1% 245/40000 [02:58<8:00:51,  1.38it/s]G: 0.16 | D: 1.88 | GP: 0.16
HW6_model</content/faces>:   1% 289/40000 [03:29<7:58:10,  1.38it/s]G: 0.19 | D: 0.85 | GP: 1.78
HW6_model</content/faces>:   1% 347/40000 [04:11<7:52:49,  1.40it/s]G: 2.21 | D: 0.73 | GP: 0.74
HW6_model</content/faces>:   1% 391/40000 [04:42<7:50:49,  1.40it/s]G: 0.59 | D: 0.63 | GP: 1.62
HW6_model</content/faces>:   1% 449/40000 [05:24<7:58:39,  1.38it/s]G: 4.47 | D: 0.14 | GP: 1.19
HW6_model</content/faces>:   1% 493/

## Inference
Use the trained model to generate anime faces!

### Load model 

In [ ]:
# Load model from modelloader
loader = ModelLoader(
    base_dir = '/content/drive/MyDrive/ML_HW6',
    name = 'HW6_model'
)

continuing from previous epoch - 40
loading from version 1.8.1


### Compress the generated images using **tar**.


In [ ]:
# Save the generated images.
os.makedirs('output', exist_ok=True)

for i in range(1000):
    noise   = torch.randn(1, 512).cuda() 
    styles  = loader.noise_to_styles(noise, trunc_psi = 0.7) 
    images  = loader.styles_to_images(styles) 
    save_image(images, f'output/{i+1}.jpg') 
  
# Compress the images.
%cd output
!tar -zcf ../images.tgz *.jpg
%cd ..

/content/output
/content
